#### includes

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import mlflow
import pytorch_lightning as pl
from pytorch_lightning.utilities.cloud_io import load as pl_load
import torch
import sys

plt.rcParams["figure.figsize"] = (18, 5)

In [3]:
sys.path.insert(0, '../../')
from lib.utils.rotequivariance_toolbox import *
from lib.steered_conv.steerable_filters import radial_steerable_filter, plot_filter
from lib.steered_conv import SteerableKernelBase
from experiments.trainer import BinaryClassifierNet
from experiments import load_dataset, parse_config, setup_model

## Constantes

In [ ]:
EXP = '/home/gaby/mlflow/mlruns/4/5160045012be4d76af5ecae8ec6f919a/artifacts/'
cfg = parse_config(EXP+'cfg_extended.yaml')
#cfg.training['dataset-file'] = '/home/gaby/These/src/Fundus/RotInvariance/3-Git/vessels.h5'
trainD, validD, testD = load_dataset(cfg)
cfg['script-arguments']['gpus']='0'

## Initialisation

In [ ]:
model = setup_model(cfg.model, old=True)
net = BinaryClassifierNet(model=model)
state_dict = pl_load(EXP+'best-val-roc.ckpt'.format(25), map_location='cuda:0')['state_dict']
net.load_state_dict(state_dict)
net.eval().cuda()
None

In [ ]:
for d in testD['MESSIDOR']:
    break

In [ ]:
THETA = 24
x = torch.from_numpy(rotate(d['x'][0].numpy(), THETA)).cuda()

In [ ]:
features = {}
def log_f(name):
    def hook(self, inputs, outputs):
        features[name] = outputs.detach().cpu().numpy()
    return hook
net.model.conv1.relu.register_forward_hook(log_f('conv1'))
net.model.conv2.relu.register_forward_hook(log_f('conv2'))
net.model.conv3.relu.register_forward_hook(log_f('conv3'))
net.model.conv4.relu.register_forward_hook(log_f('conv4'))
net.model.conv10.relu.register_forward_hook(log_f('conv10'))

None

In [ ]:
yhat = net(x)
yhat = yhat.detach().cpu().numpy()
yhat = unrotate(yhat, THETA)

## Polar Spectral Power Density

In [ ]:
_ = spectral_power(yhat, plot=True)

In [ ]:
features = {k: unrotate(v, THETA) for k,v in features.items()}

In [ ]:
spe = spectral_power(features['conv1'], plot=True, split='normed')

fig, axs = plt.subplots(2,4)
for i in range(8):
    y = spe[:, i]
    y = y/y.sum()
    x = np.arange(len(spe))
    ax = axs[i//4,i%4]
    ax.bar(x,y)
    ax.set_yscale('sqr')

In [ ]:
W = net.model.conv1.conv.weight.detach().cpu().numpy()
spe = polar_spectral_power(W, theta=24, plot='sqr', split='normed')

fig, axs = plt.subplots(2,4)
for i in range(8):
    y = spe[:, i]
    y = y/y.sum()
    x = np.arange(len(spe))
    ax = axs[i//4,i%4]
    ax.bar(x,y)
    ax.set_yscale('sqr')

In [ ]:
spe = spectral_power(features['conv2'], plot=True, split='normed')

fig, axs = plt.subplots(2,4)
for i in range(8):
    y = spe[0:len(spe)//2+1,i]
    y = y/y.sum()
    x = np.arange(0,len(spe)//2+1)
    ax = axs[i//4,i%4]
    ax.bar(x,y)
    ax.set_yscale('sqr')

In [ ]:
W = net.model.conv2.conv.weight.detach().cpu().numpy()
spe = polar_spectral_power(W, theta=24, plot='sqr', split='normed')

fig, axs = plt.subplots(2,4)
for i in range(8):
    y = spe[:, i]
    y = y/y.sum()
    x = np.arange(len(spe))
    ax = axs[i//4,i%4]
    ax.bar(x,y)
    ax.set_yscale('sqr')

In [ ]:
spe = spectral_power(features['conv3'], plot=True, split='normed')

fig, axs = plt.subplots(2,4)
for i in range(8):
    y = spe[:, i]
    y = y/y.sum()
    x = np.arange(len(spe))
    ax = axs[i//4,i%4]
    ax.bar(x,y)
    ax.set_yscale('sqr')

In [ ]:
spe = spectral_power(features['conv4'], plot=True, split='normed')

fig, axs = plt.subplots(2,4)
for i in range(8):
    y = spe[:, i]
    y = y/y.sum()
    x = np.arange(len(spe))
    ax = axs[i//4,i%4]
    ax.bar(x,y)
    ax.set_yscale('sqr')

In [ ]:
from scipy.signal import convolve2d
from scipy.ndimage import gaussian_filter

grad = np.matmul(np.linspace(-1,1,3)[:,None],np.ones((1,3)))
noise = np.random.uniform(0,1,size=(100,100))
noise = gaussian_filter(noise, 3)
noise = rotate(noise, 24)
r = np.stack([convolve2d(_, grad) for _ in noise])
r = unrotate(r, 24)
spe = spectral_power(r, plot='sqr')

In [ ]:
spe = spectral_power(features['conv10'], plot=True, split='normed')
spe = spe / spe.sum(axis=0)
print(f"Equivariant features: {(spe[0]>=.5).sum()}/{spe.shape[1]}")

## Steerable Filters

### Polar Spectral Power Density

In [ ]:
plt.rcParams.update({'font.size': 16})
F0a = np.ones((1,9))*np.arange(-4,5)[:, np.newaxis]
plot_filter(np.real(F0a), axis=False, spd=True)
F0b = np.ones((1,9))*-np.abs(np.arange(-4,5)[:, np.newaxis])
plot_filter(np.real(F0b), axis=False, spd=True)

F0 = radial_steerable_filter(15, k=0,r=6, std=1)
plot_filter(-np.real(F0), axis=False, spd=True)
F1 = radial_steerable_filter(15, k=1,r=6, std=1)
plot_filter(np.real(F1), axis=False, spd=True)
F2 = radial_steerable_filter(15, k=2,r=6, std=1)
plot_filter(np.real(F2), axis=False, spd=True)
F3 = radial_steerable_filter(15, k=3,r=6, std=1)
plot_filter(np.real(F3), axis=False, spd=True)

### Steerable Filters

In [ ]:
base = SteerableKernelBase.create_from_rk(4)
print(len(base.base))
for i, k_label in enumerate(base.kernels_label):
    plot_filter(base.base[i], spd=16, colorbar=True)
    plt.title(k_label)

### Cos Sin Combination Test

In [ ]:
base = SteerableKernelBase.create_from_rk(4)
alpha = torch.linspace(0, 2*np.pi, 4)
k_max = 5
kalpha = torch.stack([k*alpha for k in range(1,k_max+1)])
true_cos_sin_kalpha = torch.stack([torch.cos(kalpha), torch.sin(kalpha)])

In [ ]:
from lib.steered_conv.steerable_filters import cos_sin_ka, cos_sin_ka_stack
fast_cos_sin_kalpha = cos_sin_ka_stack(torch.cos(alpha), torch.sin(alpha), k_max)
diff = true_cos_sin_kalpha-fast_cos_sin_kalpha
print('Average Diff:')
pd.DataFrame({'cos':diff[0].mean(-1), 'sin': diff[0].mean(-1)}, index=list(range(1,k_max+1)))

### Steerable Test

In [5]:
base = SteerableKernelBase.create_from_rk(4)
x = torch.linspace(-2,2,5, device='cuda', dtype=torch.float)
x,y = torch.meshgrid(x,x)

K = x+y
K = K.reshape(1,1,5,5)
info = {}

W = base.approximate_weights(K, info)
W

tensor([[[-4.8384e-01, -2.3525e+05, -3.6858e+05,  1.0456e+06, -1.1118e+05,
           9.3621e+05,  1.2616e+06,  8.7349e+05,  2.9378e+05,  3.8835e+05,
          -4.7653e+05,  2.2374e+05,  2.7290e+05,  1.6086e+05,  1.6356e+05,
          -7.7553e+04, -5.2750e+04,  7.6555e+04, -2.3689e+04, -3.2193e+04,
           1.6944e+04,  7.5388e+04, -1.8686e+05,  4.0140e+04, -4.2307e+04,
           1.8755e+04,  5.1630e+04, -1.0826e+04, -2.0778e+04,  3.3235e+04,
          -2.3016e+04,  2.3101e+04, -4.3279e+04, -1.4023e+04]]],
       device='cuda:0')

**Checking weight approximation**

In [ ]:
plot_filter(K[0,0], spd=True)
plot_filter(info['y_approx'][0,0], spd=True)
print(f'mse:{info["mse"]:.3f}, r2:{info["r2"]:.3f}')


**Checking filter rotation for different steer angle $\alpha$**

In [ ]:
I = torch.zeros((1,1,7,7))
I[:,:,3,3] = 1
fig, axs = plt.subplots(1,5)
N = len(axs)-1
alphas = np.linspace(0, np.pi, N+1, endpoint=True)
for i in range(N+1):
    plot_filter(base.conv2d(I,W[0], alpha=alphas[i])[0,0], plot=axs[i])
    if i==0:
        axs[i].set_title('$\\alpha=0$')
    elif i%2 or N%2:
        axs[i].set_title('$\\alpha = \dfrac{%i \pi}{%i}$' % (i, N))
    else:
        axs[i].set_title('$\\alpha = \dfrac{%i \pi}{%i}$' % (i//2, N/2))